In [1]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", 
    na_values=['NA', '?'])

display(df)


,sepal_l,sepal_w,petal_l,petal_w,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [5]:

# Convert to numpy - Classification
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values

dummies = pd.get_dummies(df['species'], prefix='species') # Classification
species = dummies.columns
y = dummies.values
display(dummies)

,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [16]:
# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)



In [18]:
# Build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1],activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 78        
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
_________________________________________________________________


In [19]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)


model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 112 samples, validate on 38 samples
Epoch 1/1000
112/112 - 0s - loss: 1.0559 - val_loss: 1.0180
Epoch 2/1000
112/112 - 0s - loss: 0.9560 - val_loss: 0.9374
Epoch 3/1000
112/112 - 0s - loss: 0.8909 - val_loss: 0.8642
Epoch 4/1000
112/112 - 0s - loss: 0.8379 - val_loss: 0.8031
Epoch 5/1000
112/112 - 0s - loss: 0.7887 - val_loss: 0.7574
Epoch 6/1000
112/112 - 0s - loss: 0.7495 - val_loss: 0.7123
Epoch 7/1000
112/112 - 0s - loss: 0.7110 - val_loss: 0.6707
Epoch 8/1000
112/112 - 0s - loss: 0.6761 - val_loss: 0.6302
Epoch 9/1000
112/112 - 0s - loss: 0.6432 - val_loss: 0.5930
Epoch 10/1000
112/112 - 0s - loss: 0.6092 - val_loss: 0.5614
Epoch 11/1000
112/112 - 0s - loss: 0.5795 - val_loss: 0.5333
Epoch 12/1000
112/112 - 0s - loss: 0.5516 - val_loss: 0.5048
Epoch 13/1000
112/112 - 0s - loss: 0.5274 - val_loss: 0.4790
Epoch 14/1000
112/112 - 0s - loss: 0.5048 - val_loss: 0.4548
Epoch 15/1000
112/112 - 0s - loss: 0.4821 - val_loss: 0.4331
Epoch 16/1000
112/112 - 0s - loss: 0.4625 - val_l

In [31]:
# Evaluation
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 1.0
